<a href="https://colab.research.google.com/github/jackevansadl/CHEM3630/blob/main/workshop3-thermodynamicsA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Workshop 3: Thermodynamics of Gas-Phase Acidity

##Overview
Quantum chemical calculations will be used to investigate the thermodynamics of gas-phase proton abstraction reactions to determine the relative gas-phase acidities of water and other chemical species.

## Background
Reactions of acids or bases play important roles in many areas, from biology to chemistry to geology. According to the Brønsted–Lowry definition, an acid is a proton donor and a base is a proton acceptor.<a name="cite_ref-1"></a>[<sup>[1]</sup>](#cite_note-1) A key property of an acid is its strength, or propensity to donate a proton, which is characterised by the equilibrium constant 𝐾a of the acid ionisation reaction.

HA ⇌ H$^{+}$ + A$^{-}$ &emsp; &emsp; $K_a = \frac{[\mathrm{H}^+][\mathrm{A}^-]}{[\mathrm{H}\mathrm{A}]}$ (1)

where HA is the acid and A$^-$ is its conjugate base. The higher the value of 𝐾a, the greater the propensity of the acid to give up a proton and thus the stronger the acid. The acid-ionisation equilibrium constant can be calculated from the standard Gibbs free energy change ∆𝐺° for the acid ionisation reaction using:

$ΔG^\circ = -RT\ln K_a$ (2)

The standard Gibbs free energy change ∆𝐺° of a chemical reaction is the Gibbs free energy change ∆𝐺 in the case in which each reactant and product is at 1 atm pressure (for a gas) or 1 M concentration (for a solution).

Measuring the acid-ionisation equilibrium constant can be experimentally challenging, particularly for weak acids with small equilibrium constants. Fortunately, the free energy change, and thus the acid-ionisation equilibrium constant, can be accurately calculated in many cases using quantum chemistry. This practical will use a particular type of quantum chemistry called density functional theory, DFT (as implemented in Psi4<a name="cite_ref-2"></a>[<sup>[2]</sup>](#cite_note-2)) and the atomic simulation environment<a name="cite_ref-3"></a>[<sup>[3]</sup>](#cite_note-3) to calculate the thermodynamics of acid ionisation. DFT is a method that calculates the ground-state electron density and electronic energy for a specified nuclear geometry.<a name="cite_ref-4"></a>[<sup>[4]</sup>](#cite_note-4) The calculations are for molecules in the gas phase (i.e. not solvated), which can serve as a good starting point for more sophisticated calculations of the thermodynamics of acid ionisation in solution. Furthermore, gas-phase acidity and basicity play a significant role in important chemical processes, e.g. determining the propensity to form molecular ions of particular mass/charge ratios in mass spectrometry.<a name="cite_ref-5"></a>[<sup>[5]</sup>](#cite_note-5)

By varying the nuclear geometry to minimise the electronic energy in a process called geometry optimisation, the equilibrium geometry of a molecule at absolute zero temperature (0 K) can be determined. To calculate the total energy, enthalpy, or Gibbs free energy at finite (non-zero) temperature, contributions from translational, rotational, and vibrational degrees of freedom and the effect of finite temperature must be included. By calculating the electronic energy for small displacements of the atoms from the equilibrium geometry, the oscillation frequencies of the vibrational degrees of freedom (normal modes) of the molecule can be estimated using the harmonic oscillator approximation in a process called vibrational frequency analysis, from which quantum vibrational energies can be calculated. Using the electronic energy, equilibrium geometry, and vibrational frequencies from these quantum-chemical calculations, along with the molecular weight, in equations derived from statistical mechanics, the thermodynamics at finite temperature can be calculated.

## Goals and Objectives
_Overall Goals_:
1. Learn how quantum chemical calculations can be used to investigate the geometry, electronic structure, and
thermochemistry of a simple molecular system.
2. Understand the output of quantum chemical calculations.
3. Use the theory of statistical mechanics to understand how the translational, vibrational, and electronic contributions to the average energy of an isolated molecule at a given temperature is calculated from molecular parameters and the single-molecule canonical partition function.
4. Evaluate the importance of translational, rotational, and/or vibrational contributions and/or finite-temperature corrections to the thermodynamics of chemical reactions.
5. Understand the relationship between energy, enthalpy, and Gibbs free energy.
6. Discuss the gas-phase acidity of a molecular species in terms of its molecular structure.

##Specific Objectives:
1. Build and optimise molecules using quantum chemical calculations.
2. Extract the energies, geometries and vibrational frequencies of conjugate acid–base pairs from quantum chemical calculations.
2. Determine the standard Gibbs free energy change and equilibrium constant for acid ionisation in the gas phase for several acids.
3. Compare the calculated gas-phase acidities of different acids and compare the calculated values with experimentally measured values.

---
<a name="cite_note-1"></a>1. [^](#cite_ref-1)  A. Blackman, S.E. Bottle, S. Schmid, M. Mocerino, U. Wille, Chemistry (3e), Wiley, 2015.

<a name="cite_note-2"></a>2. [^](#cite_ref-2)  https://psicode.org

<a name="cite_note-3"></a>3. [^](#cite_ref-3)  https://wiki.fysik.dtu.dk/ase/

<a name="cite_note-4"></a>4. [^](#cite_ref-5)  D.S. Sholl, J.A. Steckel, Density Functional Theory: A Practical Introducton, Wiley-Interscience, 2009.

<a name="cite_note-5"></a>5. [^](#cite_ref-5)  A.G. Harrison, Mass Spectrometry Reviews 16, 201–217 (1997)

# A. Autoionisation of Water
Combined geometry optimisation and vibrational frequency calculations will be performed for water and its conjugate base, the hydroxide ion, using DFT (at the _B3LYP/6-31+G(d)_ level of theory). Use the results of these calculations to calculate the change in energy, standard enthalpy, and standard Gibbs free energy for the gas- phase acid ionisation of water at 298.15 K (25$^\circ$C).

First we need to install the DFT codes (this will take a few minutes)

In [1]:
!pip install -q condacolab

In [2]:
import condacolab
condacolab.install()

⏬ Downloading https://github.com/jaimergp/miniforge/releases/latest/download/Mambaforge-colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:28
🔁 Restarting kernel...


In [1]:
! mamba install -c anaconda intel-openmp --quiet >/dev/null 2>&1
! mamba install -c psi4 psi4 --quiet >/dev/null 2>&1
! pip install -q ase >/dev/null 2>&1
! pip install -q nglview >/dev/null 2>&1
! pip install -q 'ipywidgets>=7.6.0,<8' --force-reinstall >/dev/null 2>&1

The _ase_ (atomic simulation environment) library has routines for describing molecules and interfaces with DFT software.

In [2]:
from ase.build import molecule
from ase import Atoms

water = molecule('H2O')

d=0.97
ohminus = Atoms('OH', positions=([0-d/2,0,0],[0+d/2,0,0]), charges=[-1,0])

hplus = molecule('H')
hplus.set_initial_charges([+1])

molecules = [water, ohminus]

In [4]:
#initalise dictionaries to save the results below
h_results = {}
oh_results = {}
h2o_results = {}

Computing the energy for H$^{+}$ is doesn't require any geometry optimisation (it's only one atom).

The calculation parameters are loaded with the Psi4 object. Check out https://wiki.fysik.dtu.dk/ase/ase/calculators/psi4.html for the full list of options. **There is an error in the parameters below!**

In [7]:
from ase.calculators.psi4 import Psi4
calc = Psi4(atoms = hplus,
            method = 'b3lyp',
            memory = '500MB', # this is the default, be aware!
            basis = '6-31+G(d)',
            charge = 0,
            multiplicity=1,
            label = hplus.get_chemical_formula())

hplus.calc = calc
print(hplus.get_potential_energy())
h_results['energy'] = hplus.get_potential_energy()



c: [0]
fc: [0.0]
m: [1]
fm: [1]


ValidationError: ignored

__What is the energy for H$^+$ and in which units?__

 Answer:

For molecules we need to first optimise the atom positions to find the lowest energy positons. The calculation optimises the molecular structure through a series of geometry optimisation steps.

In [8]:
from ase.calculators.psi4 import Psi4
from ase.build import molecule
import numpy as np
from ase.optimize import QuasiNewton

optimised_molecules = []
for atoms in molecules:
  print(atoms.get_chemical_formula())
  calc = Psi4(atoms = atoms,
        method = 'b3lyp',
        memory = '500MB', # this is the default, be aware!
        basis = '6-31+G(d)',

        charge = np.sum(atoms.get_initial_charges()),
        multiplicity=1,
        label = atoms.get_chemical_formula())

  atoms.calc = calc

  QuasiNewton(atoms).run(fmax=0.0005)

  #save results
  if atoms.get_chemical_formula() == 'HO':
      oh_results['energy'] = atoms.get_potential_energy()
  if atoms.get_chemical_formula() == 'H2O':
      h2o_results['energy'] = atoms.get_potential_energy()
  print(atoms.get_potential_energy())
  optimised_molecules.append(atoms)

H2O
                Step[ FC]     Time          Energy          fmax
BFGSLineSearch:    0[  0] 06:48:35    -2079.562595        0.1365
BFGSLineSearch:    1[  2] 06:48:46    -2079.564058        0.0820
BFGSLineSearch:    2[  4] 06:48:54    -2079.564177        0.0003
-2079.5641771730634
HO
                Step[ FC]     Time          Energy          fmax
BFGSLineSearch:    0[  0] 06:49:01    -2062.532144        0.2308
BFGSLineSearch:    1[  2] 06:49:12    -2062.532703        0.0012
BFGSLineSearch:    2[  3] 06:49:17    -2062.532703        0.0000
-2062.532702910965


 __What is the convergence criteria used here (Don't forget the units)?__

 Answer:

  __What optimiser is used here and what alternatives are there?__

 Answer:

The output displayed is only a snapshot of important quantities. The files _H.dat_, _HO.dat_ and _H20.dat_ have the raw output from the _Psi4_ sofware. Check the files tab on the left side of the browser.

__Open the *H2O.dat* file and search the detailed ouput. Which point group symmetry is used here?__

Answer:

Using the harmonic oscilator approximation the vibrational degrees of freedom (normal modes) of these molecules can be computed.

In [9]:
from ase.vibrations import Vibrations
#save the results in a list for later processing
vibs = []
for atoms in optimised_molecules:
  print(atoms.get_chemical_formula())
  vib = Vibrations(atoms, name=atoms.get_chemical_formula(), delta=0.01)
  vib.clean()
  vib.run()
  vib.write_mode(0)
  vib.write_mode(-1)
  vib.summary()
  vibs.append(vib)


H2O
---------------------
  #    meV     cm^-1
---------------------
  0    6.3i     51.1i
  1    0.3i      2.8i
  2    0.2i      1.6i
  3    0.1       0.8
  4    3.3      26.7
  5    3.4      27.6
  6  206.0    1661.8
  7  463.2    3736.1
  8  478.6    3860.4
---------------------
Zero-point energy: 0.577 eV
HO
---------------------
  #    meV     cm^-1
---------------------
  0    0.0i      0.0i
  1    0.0i      0.0i
  2    0.0i      0.0i
  3    3.3      26.7
  4    3.3      26.7
  5  453.3    3656.3
---------------------
Zero-point energy: 0.230 eV


The modes as displayed include translational and rotational movements that leads low energy negative frequences, these can be ignored.

The trajectory of the 0th mode was saved (_H2O.0.traj_) and you can view this using the _view_ routine in _ase__:

In [10]:
from google.colab import output
output.enable_custom_widget_manager()

from ase.visualize import view
from ase.io import read
atoms = read('/content/H2O.0.traj', index=":")
view(atoms, viewer='ngl')

__What do the 8th vibrational mode of H2O and the 5th mode of HO relate to?__.
_You may need to clear the output of the above cell_

_Answer:_

The _thermochemistry_ routines of ASE can be used to compute several thermodynamic quantites from the normal modes using the ideal gas approximation. An example for H2O is shown below: 

In [11]:
from ase.thermochemistry import IdealGasThermo
# the 0th entry is h2o
atoms = optimised_molecules[0]
potentialenergy = atoms.get_potential_energy()
vib = vibs[0]
vib_energies = vib.get_energies()


thermo = IdealGasThermo(vib_energies=vib_energies,
                        potentialenergy=potentialenergy,
                        atoms=atoms,
                        geometry='linear',
                        symmetrynumber=2, spin=0)
G = thermo.get_gibbs_energy(temperature=298.15, pressure=101325.)
H = thermo.get_enthalpy(temperature=298.15, verbose=False)
S = thermo.get_entropy(temperature=298.15, pressure=101325., verbose=False)
h2o_results['gibbs_energy'] = G
h2o_results['enthalpy'] = H
h2o_results['entropy'] = S

Enthalpy components at T = 298.15 K:
E_pot              -2079.564 eV
E_ZPE                  0.576 eV
Cv_trans (0->T)        0.039 eV
Cv_rot (0->T)          0.026 eV
Cv_vib (0->T)          0.024 eV
(C_v -> C_p)           0.026 eV
-------------------------------
H                  -2078.875 eV

Entropy components at T = 298.15 K and P = 101325.0 Pa:
                           S               T*S
S_trans (1 bar)    0.0015019 eV/K        0.448 eV
S_rot              0.0002940 eV/K        0.088 eV
S_elec             0.0000000 eV/K        0.000 eV
S_vib              0.0002601 eV/K        0.078 eV
S (1 bar -> P)    -0.0000011 eV/K       -0.000 eV
-------------------------------------------------
S                  0.0020549 eV/K        0.613 eV

Free energy components at T = 298.15 K and P = 101325.0 Pa:
    H      -2078.875 eV
 -T*S         -0.613 eV
-----------------------
    G      -2079.487 eV


__Confirm that the average translation energy (described here as translational heat capacity Cv) is described by Eq. 9 (Appendix):__

$<ɛ_T> = -\frac{1}{q_T}(\frac{∂q_T}{∂β})_{N,V} \approx \frac{3}{2}k_BT$

In [ ]:
#answer goes here

__Compute the same thermodynamic quanties (gibbs energy, enthalpy and entropy) for OH$^-$ and H$^+$.__

In [ ]:
#answer for OH-

In [ ]:
#answer for H+ (Hint!: consider the 'geometry' parameter and set vib.energies to an empty list [])

At this point you should have three dictionaries _h\_results_, _oh\_results_ and _h2o\_results_ that includes the electronic energy, enthalpy and entropy of these species.

__Calculate the change in electronic energy $\Delta$E, the change in enthalpy $\Delta$H, the entropy contribution T$\Delta$S, and the Gibbs free energy change $\Delta$G for the reaction (Eqns 23-27):__

H$_2$O $⟶$ OH$^{-}$ + H$^+$ 


In [ ]:
#answer for deltaE

In [ ]:
#answer for deltaH

In [ ]:
#answer for deltaS

In [ ]:
#answer for deltaG

__Calculate K$_a$ and pK$_a$ at 298.15 K for the acid-ionisation reaction H2O ⇌ H$^+$ + OH$^-$ (Eqns 1 and 2 of this preamble). Discuss how this compares with the experimentally determined value and why it might differ.__

In [ ]:
#answer for Ka

In [ ]:
#answer for pKa

_Your discussion_